In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(plyr); library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following object is masked from ‘package:stats’:

    filter

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [4]:
%%R
df_all = read.csv("/var/seq_data/priming_exp/data/l2fc_table.csv")
head(df_all)

  sparsity_threshold Day Treatment       OTU log2FoldChange         p    Rank1
1               0.25  14    13C000  OTU.4204    -0.20553170 1.0000000 Bacteria
2               0.25  14    13C000   OTU.467    -0.33054855 1.0000000 Bacteria
3               0.25  14    13C000   OTU.905     0.01698885 0.9827594 Bacteria
4               0.25  14    13C000  OTU.7253     0.02799601 0.9772968 Bacteria
5               0.25  14    13C000    OTU.67    -0.60115954 1.0000000 Bacteria
6               0.25  14    13C000 OTU.10580     0.36193427 0.7248674 Bacteria
          Rank2 Rank3                Rank4 Rank5 Rank6 Rank7 Rank8 padj
1 Acidobacteria DA023                 <NA>  <NA>  <NA>  <NA>  <NA>    1
2 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
3 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
4 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
5 Acidobacteria DA023 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>    1
6 Acidobacteria

In [5]:
%%R
FDR = 0.10

df_resp = df_all %>%
    filter(padj <= FDR)

head(df_resp)

  sparsity_threshold Day Treatment       OTU log2FoldChange            p
1               0.25  14    13C000  OTU.7337       2.477516 0.0018908765
2               0.25  14    13C000   OTU.950       3.497786 0.0021470637
3               0.25  14    13C000 OTU.11380       5.169283 0.0002911960
4               0.25  14    13C000  OTU.5228       2.654885 0.0002285059
5               0.25  14    13C000   OTU.903       3.807659 0.0015749699
6               0.25  14    13C000   OTU.752       4.813177 0.0002952637
     Rank1           Rank2            Rank3              Rank4
1 Bacteria   Acidobacteria       Holophagae             iii1-8
2 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
3 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
4 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
5 Bacteria Verrucomicrobia Verrucomicrobiae Verrucomicrobiales
6 Bacteria Verrucomicrobia   Spartobacteria Chthoniobacterales
                 Rank5           Rank6          

In [6]:
%%R 

unq.resp = df_resp %>%
    group_by(OTU, Treatment) %>%
    summarise() 
unq.resp

Source: local data frame [1,000 x 2]
Groups: OTU

         OTU Treatment
1      OTU.1    13C000
2     OTU.10    13C000
3     OTU.10    13C100
4     OTU.10    13C700
5   OTU.1009    13C000
6   OTU.1009    13C100
7   OTU.1009    13C700
8    OTU.101    13C000
9   OTU.1016    13C700
10 OTU.10167    13C000
..       ...       ...


In [7]:
%%R

tax = df_resp %>%
    select(OTU, Rank2, Rank3, Rank4, Rank5) %>%
    distinct(OTU) %>%
    group_by(OTU) %>%
    arrange()

tax

Source: local data frame [574 x 5]
Groups: OTU

         OTU           Rank2                          Rank3
1      OTU.1  Actinobacteria                  Micrococcales
2     OTU.10  Proteobacteria            Alphaproteobacteria
3   OTU.1009   Bacteroidetes               Sphingobacteriia
4    OTU.101  Actinobacteria               Streptomycetales
5   OTU.1016   Acidobacteria                           RB25
6  OTU.10167   Acidobacteria Candidatus_Chloracidobacterium
7  OTU.10192   Bacteroidetes                     Cytophagia
8    OTU.102   Bacteroidetes               Sphingobacteriia
9  OTU.10213 Verrucomicrobia               Verrucomicrobiae
10 OTU.10228   Bacteroidetes                     Cytophagia
..       ...             ...                            ...
Variables not shown: Rank4 (fctr), Rank5 (fctr)


In [8]:
%%R

unq.resp.wtax = unq.resp %>%
    left_join(tax)

unq.resp.wtax

Joining by: "OTU"
Source: local data frame [1,000 x 6]
Groups: OTU

         OTU Treatment          Rank2                          Rank3
1      OTU.1    13C000 Actinobacteria                  Micrococcales
2     OTU.10    13C000 Proteobacteria            Alphaproteobacteria
3     OTU.10    13C100 Proteobacteria            Alphaproteobacteria
4     OTU.10    13C700 Proteobacteria            Alphaproteobacteria
5   OTU.1009    13C000  Bacteroidetes               Sphingobacteriia
6   OTU.1009    13C100  Bacteroidetes               Sphingobacteriia
7   OTU.1009    13C700  Bacteroidetes               Sphingobacteriia
8    OTU.101    13C000 Actinobacteria               Streptomycetales
9   OTU.1016    13C700  Acidobacteria                           RB25
10 OTU.10167    13C000  Acidobacteria Candidatus_Chloracidobacterium
..       ...       ...            ...                            ...
Variables not shown: Rank4 (fctr), Rank5 (fctr)


In [9]:
%%R

write.table(unq.resp.wtax, 
            file = "data/unq.resp.wtax.csv",
            row.names = FALSE,
            sep = ",")

In [10]:
%%R 
counts = ddply(unq.resp.wtax, .(OTU), summarize, NumSubs = length(unique(Treatment)))
head(counts)

        OTU NumSubs
1     OTU.1       1
2    OTU.10       3
3  OTU.1009       3
4   OTU.101       1
5  OTU.1016       1
6 OTU.10167       3


In [11]:
%%R
unq.one = counts %>%
    filter(NumSubs == 1) %>%
    left_join(unq.resp.wtax) %>%
    arrange(Treatment)
head(unq.one)

Joining by: "OTU"
        OTU NumSubs Treatment          Rank2               Rank3
1     OTU.1       1    13C000 Actinobacteria       Micrococcales
2   OTU.101       1    13C000 Actinobacteria    Streptomycetales
3  OTU.1029       1    13C000 Planctomycetes               OM190
4   OTU.107       1    13C000 Proteobacteria Alphaproteobacteria
5  OTU.1077       1    13C000 Planctomycetes       Phycisphaerae
6 OTU.10789       1    13C000  Bacteroidetes          Cytophagia
                 Rank4             Rank5
1       Micrococcaceae      Arthrobacter
2    Streptomycetaceae     Kitasatospora
3 uncultured_bacterium              <NA>
4          Rhizobiales Xanthobacteraceae
5      Phycisphaerales  Phycisphaeraceae
6         Cytophagales     Cytophagaceae


In [12]:
%%R

write.table(unq.one, 
            file = "data/unq.one.csv",
            row.names = FALSE,
            sep = ",")

In [13]:
%%R

cyano = df_resp %>%
    filter(Rank2 == "Cyanobacteria") %>%
    distinct(OTU)

cyano

  sparsity_threshold Day Treatment      OTU log2FoldChange            p
1               0.25  14    13C000 OTU.5011       4.843214 2.347422e-03
2               0.25  28    13C000 OTU.1155       6.386974 4.199253e-07
3               0.25  45    13C000 OTU.1629       3.364406 4.022688e-03
4               0.25  14    13C100 OTU.1992       7.281776 1.064924e-09
5               0.25  45    13C100  OTU.848       3.834780 2.403338e-05
6               0.25  45    13C700 OTU.3585       5.143983 6.994767e-05
     Rank1         Rank2   Rank3                Rank4 Rank5 Rank6 Rank7 Rank8
1 Bacteria Cyanobacteria  SM1D11 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>
2 Bacteria Cyanobacteria  SM1D11 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>
3 Bacteria Cyanobacteria MLE1-12 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>
4 Bacteria Cyanobacteria  SM1D11 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>
5 Bacteria Cyanobacteria MLE1-12 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>
6 Bacteria Cyanobacteria  SM

In [16]:
%%R
all = subset(counts, NumSubs == 3)
nrow(all)

[1] 152


In [17]:
%%R
two = subset(counts, NumSubs == 2)
nrow(two)

[1] 122
